In [2]:
from datasets import load_dataset,concatenate_datasets
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import Trainer,TrainingArguments,TrainerCallback
import numpy as np
import evaluate
import torch
from torch.utils.tensorboard import SummaryWriter
from model.my_bert import BertBiLSTMMegaCRF
from huggingface_hub import notebook_login
from transformers import AutoConfig

In [3]:
a = torch.randn(size=(2,3)).to('cuda')
print(a)
idx = torch.randperm(2)
print(idx)
print(a[idx])

tensor([[-0.5680, -0.6872,  0.1645],
        [-0.7013, -1.4670,  0.4725]], device='cuda:0')
tensor([1, 0])
tensor([[-0.7013, -1.4670,  0.4725],
        [-0.5680, -0.6872,  0.1645]], device='cuda:0')


In [4]:
notebook_login()

In [5]:
data_name = "PassbyGrocer/weibo-ner"
bert_name = "google-bert/bert-base-chinese"

In [6]:
dataset = load_dataset(data_name,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(bert_name)

In [7]:
print(dataset["train"].features["ner_tags"])

num_labels = len(dataset['train'].features['ner_tags'].feature.names)

id2label = {i: dataset['train'].features['ner_tags'].feature.names[i] for i in range(num_labels)}
label2id = {v: k for k, v in id2label.items()}

# 加载配置并设置标签映射
config = AutoConfig.from_pretrained(bert_name, num_labels=num_labels, id2label=id2label, label2id=label2id)

# 初始化自定义模型
model = BertBiLSTMMegaCRF.from_pretrained(bert_name, config=config)

Sequence(feature=ClassLabel(names=['O', 'B-GPE.NAM', 'I-GPE.NAM', 'B-GPE.NOM', 'I-GPE.NOM', 'B-LOC.NAM', 'I-LOC.NAM', 'B-LOC.NOM', 'I-LOC.NOM', 'B-ORG.NAM', 'I-ORG.NAM', 'B-ORG.NOM', 'I-ORG.NOM', 'B-PER.NAM', 'I-PER.NAM', 'B-PER.NOM', 'I-PER.NOM'], id=None), length=-1, id=None)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
label_list = dataset["train"].features["ner_tags"].feature.names
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding='max_length', is_split_into_words=True,max_length=64)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

In [9]:
metric = evaluate.load("seqeval")

In [10]:
writer = SummaryWriter(log_dir="./logs")
label_list = dataset["train"].features["ner_tags"].feature.names  # 获取标签名称列表
s = [i for i in range(len(label_list))]

def compute_metrics(p):
    # 获取 logits，并将它们转换为预测的标签索引
    predictions = np.argmax(p.predictions, axis=2)
    references = p.label_ids

    # 转换为字符串标签，并过滤掉 -100
    true_predictions = [
    [label_list[pred] if label in s else label_list[0] for (pred, label) in zip(prediction, reference)]
    for prediction, reference in zip(predictions, references)
    ]

    true_labels = [
    [label_list[label] if label in s else label_list[0] for (pred, label) in zip(prediction, reference)]
    for prediction, reference in zip(predictions, references)
    ]

# 计算 seqeval 指标并过滤无效标签
    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    # 检查并设置缺失标签的 F1 和 Recall 为 0
    for key, value in results.items():
        if value is None:
            results[key] = 1

    return {
        "precision": results.get("overall_precision", 1),
        "recall": results.get("overall_recall", 1),
        "f1": results.get("overall_f1", 1),
        "accuracy": results.get("overall_accuracy", 1),
    }

In [11]:
class TensorBoardCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        # 将每次日志的损失值记录到 TensorBoard
        for k, v in logs.items():
            if "loss" in k:
                writer.add_scalar(f"Loss/{k}", v, global_step=state.global_step)

In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs",                # 指定日志目录
    logging_strategy="steps",            # 设置日志记录策略（这里按步记录）
    logging_steps=10,                    # 每 10 步记录一次日志
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    fp16=False,
    push_to_hub=True,
    hub_model_id="PassbyGrocer/bert_bilstm_mega_crf-ner-weibo"
)

E:\ProgramData\anaconda3\envs\hf\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[TensorBoardCallback()],
    optimizers=(optimizer, None)
)

trainer.train()

C:\Users\sjsun\AppData\Local\Temp\ipykernel_12340\1315859427.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.330400,0.262487,0.000000,0.000000,0.000000,0.953996
2,0.179900,0.144036,0.530055,0.610063,0.567251,0.976011
3,0.130300,0.117210,0.586402,0.650943,0.616990,0.979263
4,0.096100,0.107803,0.574742,0.701258,0.631728,0.979089
5,0.055400,0.116382,0.586035,0.738994,0.653686,0.978625
6,0.056100,0.119603,0.587940,0.735849,0.653631,0.978218
7,0.042100,0.130824,0.586207,0.694969,0.635971,0.977695
8,0.035600,0.128349,0.581218,0.720126,0.643258,0.976708


E:\ProgramData\anaconda3\envs\hf\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_results = trainer.predict(tokenized_datasets["test"])
# 输出测试结果
print("测试集上的结果:", test_results.metrics)

In [ ]:
trainer.push_to_hub()